# Uso de paragraph vector
En este ejemplo se entrenará un modelo utilizando *[paragraph vector](https://cs.stanford.edu/~quocle/paragraph_vector.pdf)*, también conocido como *Doc2Vec*. En concreto usaremos la implementación proporcionada por [gensim](https://radimrehurek.com/gensim/models/doc2vec.html).
Los pasos necesarios para obtener el vector de *features* para un texto es:
1. Cargar los textos a procesar
2. Entrenar el modelo de Doc2Vec
3. Obtener la codificación para los textos


## PASO 1: Carga de los textos a procesar
Cargaremos el csv de datos que se encuentra en la ruta '../datasets/Recursos_turisticos.csv', en el cual se almacena en cada linea los datos de un recursos turístico. Para este ejemplo usaremso el campo 'strDescripcion'.

In [1]:
from google.colab import files
uploaded = files.upload()

Saving Recursos_turisticos.csv to Recursos_turisticos.csv


In [3]:
import pandas as pd

recursos = pd.read_csv('Recursos_turisticos.csv', sep='|')
recursos.head(5)

documentos = recursos['strDescripcion']
print("Numero de documentos:", len(documentos))
print("\n\nEjemplo de un documento:\n\n", documentos[1])

Numero de documentos: 601


Ejemplo de un documento:

 Enclavado en una situación privilegiada con fantásticas vistas a la sierra, y a menos de 2 Km. del centro de la ciudad de Ronda y 45 Km. de la Costa del Sol, es un lugar ideal para disfrutar de la oferta cultural y gastronómica además del patrimonio histórico-artístico de la ciudad, pudiendo combinarla con actividades en su entorno natural como la Sierra de las Nieves (reserva de la biosfera). Su decoración de estilo moderno y actual ofrece un ambiente selecto y acogedor. En nuestro restaurante podrán degustar los platos de nuestro cocinero Antonio Castro Aguilar además de una exquisita carta de vinos..


Una vez cargado los datos del csv, añadimos estos a la variable `tagged_data` en la que se almacenará el texto *tokenizado*, es decir partido por palabras, y una etiqueta, en este caso el numero del documentos. 
Para el *tokenizado* se usa la función `word_tokenize` proporcionada por la biblioteca `nltk`.

In [4]:
from gensim.models.doc2vec import TaggedDocument
from nltk.tokenize import word_tokenize
tagged_data = [TaggedDocument(words=word_tokenize(_d.decode('utf-8') .lower()), tags=[str(i)]) for i, _d in enumerate(documentos)]

AttributeError: ignored

## PASO 2: Entrenamiento del modelo de Doc2Vec
Los modelos de Doc2Vec requieren para su entrenamiento el fijar los valores para una serie de hiperparámetros, que son:
* **Longitud del vector**: longitud del vector de *features* que se generará por el modelo
* **Ventana**: tamaño de la ventana de palabras que usará el modelo
* **Frecuencia mínima**: frecuencia mínima de apariciones que debe tener un término para que sea tomado en consideración
* ***Learning rate***: valor inicial para el *learning rate* en el proceso de entrenamiento
* ***Mínimo learning rate***: valor mínimo de *learning rate* que se usará en el proceso de entrenamiento
* ***Epoch***: número de iteraciones máximas del proceso de entrenamiento

En un proceso de entrenamiento para producción es necesario realizar un proceso de busquedas de los valores a usar, recomendandose el uso de [busqueda aleatoria](http://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf). 

En este ejemplo para evitar la demora de la búsqueda de hiperparámetros se asignarán unos valores por defecto, que son:
* **Longitud del vector**: 5
* **Ventana**: 10
* **Frecuencia mínima**: 5
* ***Learning rate***: 0.025
* ***Mínimo learning rate***: 0.025
* ***Epoch***: 100

In [0]:
from gensim.models.doc2vec import Doc2Vec

#Configuración
#-------------
__longitud_vector = 5
__ventana = 10
__frecuencia_minima = 5
__learning_rate = 0.025
__min_learning_rate = 0.025
__epoch = 100
#---------------------

                
tagged_data = [TaggedDocument(words=word_tokenize(_d.decode('utf-8') .lower()), tags=[str(i)]) for i, _d in enumerate(documentos)]
model = Doc2Vec(vector_size=__longitud_vector, window=__ventana, min_count=__frecuencia_minima, workers=11,alpha=__learning_rate, min_alpha=__min_learning_rate, epochs=__epoch)
model.build_vocab(tagged_data)
model.train(tagged_data, epochs=model.epochs, total_examples=model.corpus_count)

#Recorrido de todos los documentos con los que se ha entrenado
for i in range(20):
    print model.docvecs[i].tolist()
    

[0.41015326976776123, -0.6406916379928589, -1.5952590703964233, 3.8950421810150146, 0.6677010655403137]
[1.6461111307144165, 1.719222903251648, -0.04054301977157593, 1.8220890760421753, 0.950815737247467]
[-2.7238411903381348, 2.4118847846984863, -0.9959932565689087, 3.8765854835510254, 1.7178045511245728]
[-2.682331085205078, 1.3908506631851196, 2.109365463256836, 4.925195693969727, 3.41788387298584]
[-0.16584119200706482, 0.7495830655097961, 3.441272497177124, 2.577284336090088, 1.3693571090698242]
[0.24096928536891937, 4.7898149490356445, 0.1739889532327652, 2.8192317485809326, 0.18219101428985596]
[2.6826171875, 1.635993242263794, -0.18182489275932312, -0.61750727891922, 0.7514306902885437]
[-1.5459681749343872, 4.541253566741943, -0.2771131992340088, 1.0310827493667603, 1.0837702751159668]
[0.3874787390232086, -1.9611867666244507, 2.082097291946411, 1.6078522205352783, 1.858471393585205]
[1.1283841133117676, 2.493962526321411, -1.6594595909118652, 4.6127166748046875, -0.3505018651

## PASO 3: Obtener la codificación para los textos
Existen dos caminos alternativos para la obtención de las *features* de los documentos, dependiendo si son documentos que se han usado para el entrenamiento y se encuentrán en el modelo o si son documentos nuevos.

### Obtener *features* para los textos de entrenamiento
En el siguiente ejemplo se obtienen las *features* de los primeros veinte documentos usados en el entrenamiento.

In [0]:
#Recorrido de los 20 primeros documentos con los que se ha entrenado
for i in range(20):
    print model.docvecs[i].tolist()

[0.41015326976776123, -0.6406916379928589, -1.5952590703964233, 3.8950421810150146, 0.6677010655403137]
[1.6461111307144165, 1.719222903251648, -0.04054301977157593, 1.8220890760421753, 0.950815737247467]
[-2.7238411903381348, 2.4118847846984863, -0.9959932565689087, 3.8765854835510254, 1.7178045511245728]
[-2.682331085205078, 1.3908506631851196, 2.109365463256836, 4.925195693969727, 3.41788387298584]
[-0.16584119200706482, 0.7495830655097961, 3.441272497177124, 2.577284336090088, 1.3693571090698242]
[0.24096928536891937, 4.7898149490356445, 0.1739889532327652, 2.8192317485809326, 0.18219101428985596]
[2.6826171875, 1.635993242263794, -0.18182489275932312, -0.61750727891922, 0.7514306902885437]
[-1.5459681749343872, 4.541253566741943, -0.2771131992340088, 1.0310827493667603, 1.0837702751159668]
[0.3874787390232086, -1.9611867666244507, 2.082097291946411, 1.6078522205352783, 1.858471393585205]
[1.1283841133117676, 2.493962526321411, -1.6594595909118652, 4.6127166748046875, -0.3505018651

### Obtener features para nuevos textos
Gracias al método `infer_vector` del modelo se permite obtener las *features* de un nuevo texto, el cual se pasa al método como una lista de palabras.

In [0]:
featuresNuevoTexto = model.infer_vector(["Esto", "es", "un", "texto", "desconocido", "para", "el", "modelo"])
print featuresNuevoTexto

[0.24811527 0.14234225 0.06730658 0.3178042  0.35057995]
